In [ ]:
import tensorflow as tf
from datetime import datetime
import time
import math
 
slim=tf.contrib.slim
#产生截断的正态分布
trunc_normal =lambda stddev:tf.truncated_normal_initializer(0.0,stddev)
parameters =[] #储存参数
 
 
#why？为什么要定义这个函数？
#因为若事先定义好slim.conv2d各种默认参数，包括激活函数、标准化器，后面定义卷积层将会非常容易：
# 1.代码整体美观
# 2.网络设计的工作量会大大减轻
 
def inception_v3_arg_scope(weight_decay=0.00004,
                           stddev=0.1,
                           batch_norm_var_collection='moving_vars'):
    """
    #定义inception_v3_arg_scope(),
    #用来生成网络中经常用到的函数的默认参数（卷积的激活函数、权重初始化方式、标准化器等）
    :param weight_decay: 权值衰减系数
    :param stddev: 标准差
    :param batch_norm_var_collection:
    :return:
    """
    batch_norm_params={
        'decay':0.9997, #衰减系数decay
        'epsilon':0.001, #极小值
        'updates_collections':tf.GraphKeys.UPDATE_OPS,
        'variables_collections':{
            'beta':None,
            'gamma':None,
            'moving_mean':[batch_norm_var_collection],
            'moving_variance':[batch_norm_var_collection],
        }
 
    }
 
    with slim.arg_scope([slim.conv2d,slim.fully_connected],
                        weights_regularizer=slim.l2_regularizer(weight_decay)):
        """
        slim.arg_scope()是一个非常有用的工具，可以给函数的参数自动赋予某些默认值
        
        例如：
        slim.arg_scope([slim.conv2d,slim.fully_connected],weights_regularizer=slim.l2_regularizer(weight_decay)):
        会对[slim.conv2d,slim.fully_connected]这两个函数的参数自动赋值，
        将参数weights_regularizer的默认值设为slim.l2_regularizer(weight_decay)
        
        备注：使用了slim.arg_scope后就不需要每次重复设置参数，只需在修改时设置即可。
        """
        # 设置默认值：对slim.conv2d函数的几个参数赋予默认值
        with slim.arg_scope(
            [slim.conv2d],
            weights_initializer=tf.truncated_normal_initializer(stddev=stddev), #权重初始化
            activation_fn=tf.nn.relu,  #激励函数
            normalizer_fn=slim.batch_norm,  #标准化器
            normalizer_params=batch_norm_params ) as sc: #normalizer_params标准化器的参数
 
            return sc  #返回定义好的scope
 
 
def inception_V3_base(input,scope=None):
 
    end_points= {}
    # 第一部分--基础部分：卷积和池化交错
    with tf.variable_scope(scope,'inception_V3',[input]):
        with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d],
                            stride=1,padding='VALID'):
            net1=slim.conv2d(input,32,[3,3],stride=2,scope='conv2d_1a_3x3')
            net2 = slim.conv2d(net1, 32, [3, 3],scope='conv2d_2a_3x3')
            net3 = slim.conv2d(net2, 64, [3, 3], padding='SAME',
                               scope='conv2d_2b_3x3')
            net4=slim.max_pool2d(net3,[3,3],stride=2,scope='maxPool_3a_3x3')
            net5 = slim.conv2d(net4, 80, [1, 1], scope='conv2d_4a_3x3')
            net6 = slim.conv2d(net5, 192, [3, 3], padding='SAME',
                               scope='conv2d_4b_3x3')
            net = slim.max_pool2d(net6, [3, 3], stride=2, scope='maxPool_5a_3x3')
 
    #第二部分--Inception模块组：inception_1\inception_2\inception_2
    with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d],
                            stride=1,padding='SAME'):
#inception_1：第一个模块组（共含3个inception_module）
        #inception_1_m1: 第一组的1号module
        with tf.variable_scope('inception_1_m1'):
            with tf.variable_scope('Branch_0'):
                branch_0=slim.conv2d(net,64,[1,1],scope='conv2d_0a_1x1')
            with tf.variable_scope('Branch_1'):
                branch1_1 = slim.conv2d(net, 48, [1, 1], scope='conv2d_1a_1x1')
                branch1_2 = slim.conv2d(branch1_1, 64, [5, 5],
                                        scope='conv2d_1b_5x5')
            with tf.variable_scope('Branch_2'):
                branch2_1 = slim.conv2d(net, 64, [1, 1], scope='conv2d_2a_1x1')
                branch2_2 = slim.conv2d(branch2_1, 96, [3, 3],
                                        scope='conv2d_2b_3x3')
                branch2_3 = slim.conv2d(branch2_2, 96, [3, 3],
                                        scope='conv2d_2c_3x3')
            with tf.variable_scope('Branch_3'):
                branch3_1 = slim.avg_pool2d(net, [3, 3], scope='avgPool_3a_3x3')
                branch3_2 = slim.conv2d(branch3_1, 32, [1, 1],
                                        scope='conv2d_3b_1x1')
            #使用concat将4个分支的输出合并到一起（在第三个维度合并，即输出通道上合并）
            net=tf.concat([branch_0,branch1_2,branch2_3,branch3_2],3)
 
        # inception_1_m2: 第一组的 2号module
        with tf.variable_scope('inception_1_m2'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 64, [1, 1], scope='conv2d_0a_1x1')
            with tf.variable_scope('Branch_1'):
                branch1_1 = slim.conv2d(net, 48, [1, 1], scope='conv2d_1a_1x1')
                branch1_2 = slim.conv2d(branch1_1, 64, [5, 5],
                                            scope='conv2d_1b_5x5')
            with tf.variable_scope('Branch_2'):
                branch2_1 = slim.conv2d(net, 64, [1, 1], scope='conv2d_2a_1x1')
                branch2_2 = slim.conv2d(branch2_1, 96, [3, 3],
                                            scope='conv2d_2b_3x3')
                branch2_3 = slim.conv2d(branch2_2, 96, [3, 3],
                                            scope='conv2d_2c_3x3')
            with tf.variable_scope('Branch_3'):
                branch3_1 = slim.avg_pool2d(net, [3, 3], scope='avgPool_3a_3x3')
                branch3_2 = slim.conv2d(branch3_1, 64, [1, 1],
                                            scope='conv2d_3b_1x1')
            # 使用concat将4个分支的输出合并到一起（在第三个维度合并，即输出通道上合并）
            net = tf.concat([branch_0, branch1_2, branch2_3, branch3_2], 3)
 
        # inception_1_m2: 第一组的 3号module
        with tf.variable_scope('inception_1_m3'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 64, [1, 1], scope='conv2d_0a_1x1')
            with tf.variable_scope('Branch_1'):
                branch1_1 = slim.conv2d(net, 48, [1, 1], scope='conv2d_1a_1x1')
                branch1_2 = slim.conv2d(branch1_1, 64, [5, 5],
                                            scope='conv2d_1b_5x5')
            with tf.variable_scope('Branch_2'):
                branch2_1 = slim.conv2d(net, 64, [1, 1], scope='conv2d_2a_1x1')
                branch2_2 = slim.conv2d(branch2_1, 96, [3, 3],
                                            scope='conv2d_2b_3x3')
                branch2_3 = slim.conv2d(branch2_2, 96, [3, 3],
                                            scope='conv2d_2c_3x3')
            with tf.variable_scope('Branch_3'):
                branch3_1 = slim.avg_pool2d(net, [3, 3], scope='avgPool_3a_3x3')
                branch3_2 = slim.conv2d(branch3_1, 64, [1, 1],
                                            scope='conv2d_3b_1x1')
            # 使用concat将4个分支的输出合并到一起（在第三个维度合并，即输出通道上合并）
            net = tf.concat([branch_0, branch1_2, branch2_3, branch3_2], 3)
 
#inception_2：第2个模块组（共含5个inception_module）
        # inception_2_m1: 第2组的 1号module
        with tf.variable_scope('inception_2_m1'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 384, [3, 3],stride=2,
                                       padding='VALID',scope='conv2d_0a_3x3')
            with tf.variable_scope('Branch_1'):
                branch1_1 = slim.conv2d(net, 64, [1, 1], scope='conv2d_1a_1x1')
                branch1_2 = slim.conv2d(branch1_1, 96, [3, 3],
                                            scope='conv2d_1b_3x3')
                branch1_3 = slim.conv2d(branch1_2, 96, [3, 3],
                                        stride=2,
                                        padding='VALID',
                                        scope='conv2d_1c_3x3')
            with tf.variable_scope('Branch_2'):
                branch2_1 = slim.max_pool2d(net, [3, 3],
                                        stride=2,
                                        padding='VALID',
                                        scope='maxPool_2a_3x3')
 
            # 使用concat将4个分支的输出合并到一起（在第三个维度合并，即输出通道上合并）
            net = tf.concat([branch_0, branch1_3, branch2_1], 3)
 
        # inception_2_m2: 第2组的 2号module
        with tf.variable_scope('inception_2_m2'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 192, [1, 1],scope='conv2d_0a_1x1')
            with tf.variable_scope('Branch_1'):
                branch1_1 = slim.conv2d(net, 128, [1, 1], scope='conv2d_1a_1x1')
                branch1_2 = slim.conv2d(branch1_1, 128, [1, 7],
                                            scope='conv2d_1b_1x7')
                branch1_3 = slim.conv2d(branch1_2, 128, [7, 1],
                                        scope='conv2d_1c_7x1')
            with tf.variable_scope('Branch_2'):
                branch2_1 = slim.conv2d(net, 128, [1, 1], scope='conv2d_2a_1x1')
                branch2_2 = slim.conv2d(branch2_1, 128, [7, 1],
                                        scope='conv2d_2b_7x1')
                branch2_3 = slim.conv2d(branch2_2, 128, [1, 7],
                                        scope='conv2d_2c_1x7')
                branch2_4 = slim.conv2d(branch2_3, 128, [7, 1],
                                        scope='conv2d_2d_7x1')
                branch2_5 = slim.conv2d(branch2_4, 128, [1, 7],
                                        scope='conv2d_2e_1x7')
            with tf.variable_scope('Branch_3'):
                branch3_1 = slim.avg_pool2d(net, [3, 3], scope='avgPool_3a_3x3')
                branch3_2 = slim.conv2d(branch3_1, 192, [1, 1],
                                        scope='conv2d_3b_1x1')
 
            # 使用concat将4个分支的输出合并到一起（在第三个维度合并，即输出通道上合并）
            net = tf.concat([branch_0, branch1_3, branch2_5,branch3_2], 3)
 
 
# inception_2_m3: 第2组的 3号module
        with tf.variable_scope('inception_2_m3'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 192, [1, 1],scope='conv2d_0a_1x1')
            with tf.variable_scope('Branch_1'):
                branch1_1 = slim.conv2d(net, 160, [1, 1], scope='conv2d_1a_1x1')
                branch1_2 = slim.conv2d(branch1_1, 160, [1, 7],
                                            scope='conv2d_1b_1x7')
                branch1_3 = slim.conv2d(branch1_2, 192, [7, 1],
                                        scope='conv2d_1c_7x1')
            with tf.variable_scope('Branch_2'):
                branch2_1 = slim.conv2d(net, 160, [1, 1], scope='conv2d_2a_1x1')
                branch2_2 = slim.conv2d(branch2_1, 160, [7, 1],
                                        scope='conv2d_2b_7x1')
                branch2_3 = slim.conv2d(branch2_2, 160, [1, 7],
                                        scope='conv2d_2c_1x7')
                branch2_4 = slim.conv2d(branch2_3, 160, [7, 1],
                                        scope='conv2d_2d_7x1')
                branch2_5 = slim.conv2d(branch2_4, 192, [1, 7],
                                        scope='conv2d_2e_1x7')
            with tf.variable_scope('Branch_3'):
                branch3_1 = slim.avg_pool2d(net, [3, 3], scope='avgPool_3a_3x3')
                branch3_2 = slim.conv2d(branch3_1, 192, [1, 1],
                                        scope='conv2d_3b_1x1')
 
            # 使用concat将4个分支的输出合并到一起（在第三个维度合并，即输出通道上合并）
            net = tf.concat([branch_0, branch1_3, branch2_5,branch3_2], 3)
 
# inception_2_m4: 第2组的 4号module
        with tf.variable_scope('inception_2_m4'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 192, [1, 1],scope='conv2d_0a_1x1')
            with tf.variable_scope('Branch_1'):
                branch1_1 = slim.conv2d(net, 160, [1, 1], scope='conv2d_1a_1x1')
                branch1_2 = slim.conv2d(branch1_1, 160, [1, 7],
                                            scope='conv2d_1b_1x7')
                branch1_3 = slim.conv2d(branch1_2, 192, [7, 1],
                                        scope='conv2d_1c_7x1')
            with tf.variable_scope('Branch_2'):
                branch2_1 = slim.conv2d(net, 160, [1, 1], scope='conv2d_2a_1x1')
                branch2_2 = slim.conv2d(branch2_1, 160, [7, 1],
                                        scope='conv2d_2b_7x1')
                branch2_3 = slim.conv2d(branch2_2, 160, [1, 7],
                                        scope='conv2d_2c_1x7')
                branch2_4 = slim.conv2d(branch2_3, 160, [7, 1],
                                        scope='conv2d_2d_7x1')
                branch2_5 = slim.conv2d(branch2_4, 192, [1, 7],
                                        scope='conv2d_2e_1x7')
            with tf.variable_scope('Branch_3'):
                branch3_1 = slim.avg_pool2d(net, [3, 3], scope='avgPool_3a_3x3')
                branch3_2 = slim.conv2d(branch3_1, 192, [1, 1],
                                        scope='conv2d_3b_1x1')
 
            # 使用concat将4个分支的输出合并到一起（在第三个维度合并，即输出通道上合并）
            net = tf.concat([branch_0, branch1_3, branch2_5,branch3_2], 3)
 
# inception_2_m5: 第2组的 5号module
        with tf.variable_scope('inception_2_m5'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 192, [1, 1],scope='conv2d_0a_1x1')
            with tf.variable_scope('Branch_1'):
                branch1_1 = slim.conv2d(net, 160, [1, 1], scope='conv2d_1a_1x1')
                branch1_2 = slim.conv2d(branch1_1, 160, [1, 7],
                                            scope='conv2d_1b_1x7')
                branch1_3 = slim.conv2d(branch1_2, 192, [7, 1],
                                        scope='conv2d_1c_7x1')
            with tf.variable_scope('Branch_2'):
                branch2_1 = slim.conv2d(net, 160, [1, 1], scope='conv2d_2a_1x1')
                branch2_2 = slim.conv2d(branch2_1, 160, [7, 1],
                                        scope='conv2d_2b_7x1')
                branch2_3 = slim.conv2d(branch2_2, 160, [1, 7],
                                        scope='conv2d_2c_1x7')
                branch2_4 = slim.conv2d(branch2_3, 160, [7, 1],
                                        scope='conv2d_2d_7x1')
                branch2_5 = slim.conv2d(branch2_4, 192, [1, 7],
                                        scope='conv2d_2e_1x7')
            with tf.variable_scope('Branch_3'):
                branch3_1 = slim.avg_pool2d(net, [3, 3], scope='avgPool_3a_3x3')
                branch3_2 = slim.conv2d(branch3_1, 192, [1, 1],
                                        scope='conv2d_3b_1x1')
 
            # 使用concat将4个分支的输出合并到一起（在第三个维度合并，即输出通道上合并）
            net = tf.concat([branch_0, branch1_3, branch2_5,branch3_2], 3)
        #将inception_2_m5存储到end_points中，作为Auxiliary Classifier辅助模型的分类
        end_points['inception_2_m5']=net
 
# 第3组
# inception_3_m1: 第3组的 1号module
        with tf.variable_scope('inception_3_m1'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 192, [1, 1],scope='conv2d_0a_1x1')
                branch_0 = slim.conv2d(branch_0,320, [3, 3],
                                       stride=2,
                                       padding='VALID',
                                       scope='conv2d_0b_3x3')
            with tf.variable_scope('Branch_1'):
                branch1_1 = slim.conv2d(net, 192, [1, 1], scope='conv2d_1a_1x1')
                branch1_2 = slim.conv2d(branch1_1, 192, [1, 7],
                                            scope='conv2d_1b_1x7')
                branch1_3 = slim.conv2d(branch1_2, 192, [7, 1],
                                        scope='conv2d_1c_7x1')
                branch1_4 = slim.conv2d(branch1_3, 192, [3, 3],
                                        stride=2,
                                        padding='VALID',
                                        scope='conv2d_1c_3x3')
            with tf.variable_scope('Branch_2'):
                branch2_1 = slim.max_pool2d(net, [3, 3],
                                            stride=2,
                                            padding='VALID',
                                            scope='maxPool_3a_3x3')
 
            # 使用concat将4个分支的输出合并到一起（在第三个维度合并，即输出通道上合并）
            net = tf.concat([branch_0, branch1_4, branch2_1], 3)
 
# inception_3_m2: 第3组的 2号module
        with tf.variable_scope('inception_3_m2'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 320, [1, 1],scope='conv2d_0a_1x1')
            with tf.variable_scope('Branch_1'):
                branch1_1 = slim.conv2d(net, 384, [1, 1], scope='conv2d_1a_1x1')
#特殊
                branch1_2 = tf.concat([
                    slim.conv2d(branch1_1, 384, [1, 3], scope='conv2d_1a_1x3'),
                    slim.conv2d(branch1_1, 384, [3, 1], scope='conv2d_1a_3x1')
                    ], 3)
            with tf.variable_scope('Branch_2'):
                branch2_1 = slim.conv2d(net, 488, [1, 1], scope='conv2d_2a_1x1')
                branch2_2 = slim.conv2d(branch2_1, 384, [3, 3],
                                        scope='conv2d_2b_3x3')
                branch2_3 = tf.concat([
                    slim.conv2d(branch2_2, 384, [1, 3], scope='conv2d_1a_1x3'),
                    slim.conv2d(branch2_2, 384, [3, 1], scope='conv2d_1a_3x1')
                    ], 3)
            with tf.variable_scope('Branch_3'):
                branch3_1 = slim.avg_pool2d(net, [3, 3], scope='avgPool_3a_3x3')
                branch3_2 = slim.conv2d(branch3_1, 192, [1, 1],
                                        scope='conv2d_3b_1x1')
 
            # 使用concat将4个分支的输出合并到一起（在第三个维度合并，即输出通道上合并）
            net = tf.concat([branch_0, branch1_2, branch2_3,branch3_2], 3)
 
# inception_3_m3: 第3组的 3号module
        with tf.variable_scope('inception_3_m3'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 320, [1, 1],scope='conv2d_0a_1x1')
            with tf.variable_scope('Branch_1'):
                branch1_1 = slim.conv2d(net, 384, [1, 1], scope='conv2d_1a_1x1')
#特殊
                branch1_2 = tf.concat([
                    slim.conv2d(branch1_1, 384, [1, 3], scope='conv2d_1a_1x3'),
                    slim.conv2d(branch1_1, 384, [3, 1], scope='conv2d_1a_3x1')
                    ], 3)
            with tf.variable_scope('Branch_2'):
                branch2_1 = slim.conv2d(net, 488, [1, 1], scope='conv2d_2a_1x1')
                branch2_2 = slim.conv2d(branch2_1, 384, [3, 3],
                                        scope='conv2d_2b_3x3')
                branch2_3 = tf.concat([
                    slim.conv2d(branch2_2, 384, [1, 3], scope='conv2d_1a_1x3'),
                    slim.conv2d(branch2_2, 384, [3, 1], scope='conv2d_1a_3x1')
                    ], 3)
            with tf.variable_scope('Branch_3'):
                branch3_1 = slim.avg_pool2d(net, [3, 3], scope='avgPool_3a_3x3')
                branch3_2 = slim.conv2d(branch3_1, 192, [1, 1],
                                        scope='conv2d_3b_1x1')
 
            # 使用concat将4个分支的输出合并到一起（在第三个维度合并，即输出通道上合并）
            net = tf.concat([branch_0, branch1_2, branch2_3,branch3_2], 3)
 
        return net,end_points
##############################  卷积部分完成  ########################################
 
 
#第三部分：全局平均池化、softmax、Auxiliary Logits
def inception_v3(input,
                 num_classes=1000,
                 is_training=True,
                 dropout_keep_prob=0.8,
                 prediction_fn=slim.softmax,
                 spatial_squeeze=True,
                 reuse=None,
                 scope='inceptionV3'):
    with tf.variable_scope(scope,'inceptionV3',[input,num_classes],
                       reuse=reuse) as scope:
        with slim.arg_scope([slim.batch_norm,slim.dropout],
                            is_training=is_training):
             net,end_points=inception_V3_base(input,scope=scope)
 
            #Auxiliary  Logits
             with slim.arg_scope([slim.conv2d,slim.max_pool2d,slim.avg_pool2d],
                            stride=1,padding='SAME'):
                aux_logits=end_points['inception_2_m5']
                with tf.variable_scope('Auxiliary_Logits'):
                    aux_logits=slim.avg_pool2d(
                        aux_logits,[5,5],stride=3,padding='VALID',
                        scope='AvgPool_1a_5x5' )
                    aux_logits=slim.conv2d(aux_logits,128,[1,1],
                                       scope='conv2d_1b_1x1')
                    aux_logits=slim.conv2d(aux_logits,768,[5,5],
                                       weights_initializer=trunc_normal(0.01),
                                       padding='VALID',
                                       scope='conv2d_2a_5x5')
                    aux_logits = slim.conv2d(aux_logits, num_classes, [1, 1],
                                         activation_fn=None,
                                         normalizer_fn=None,
                                         weights_initializer=trunc_normal(0.001),
                                         scope='conv2d_2b_1x1')
                    if spatial_squeeze:
                        aux_logits =tf.squeeze(aux_logits,[1,2],name='SpatialSqueeze')
                    end_points['Auxiliary_Logits']=aux_logits
 
             with tf.variable_scope('Logits'):
                 net=slim.avg_pool2d(net,[8,8],padding='VALID',
                                scope='avgPool_1a_8x8')
                 net=slim.dropout(net,keep_prob=dropout_keep_prob,
                             scope='dropout_1b')
                 end_points['PreLogits']=net
                 logits=slim.conv2d(net,num_classes,[1,1],activation_fn=None,
                               normalizer_fn=None,
                               scope='conv2d_1c_1x1')
                 if spatial_squeeze:
                     logits=tf.squeeze(logits,[1,2],name='SpatialSqueeze')
             end_points['Logits']=logits
             end_points['Predictions']=prediction_fn(logits,scope='Predictions')
 
    return logits,end_points
 
########################### 构建完成
 
 
def time_compute(session, target, info_string):
    num_batch = 100 #100
    num_step_burn_in = 10  # 预热轮数，头几轮迭代有显存加载、cache命中等问题可以因此跳过
    total_duration = 0.0  # 总时间
    total_duration_squared = 0.0
    for i in range(num_batch + num_step_burn_in):
        start_time = time.time()
        _ = session.run(target )
        duration = time.time() - start_time
        if i >= num_step_burn_in:
            if i % 10 == 0:  # 每迭代10次显示一次duration
                print("%s: step %d,duration=%.5f " % (datetime.now(), i - num_step_burn_in, duration))
            total_duration += duration
            total_duration_squared += duration * duration
    time_mean = total_duration / num_batch
    time_variance = total_duration_squared / num_batch - time_mean * time_mean
    time_stddev = math.sqrt(time_variance)
    # 迭代完成，输出
    print("%s: %s across %d steps,%.3f +/- %.3f sec per batch " %
          (datetime.now(), info_string, num_batch, time_mean, time_stddev))
 
 
def main():
    with tf.Graph().as_default():
        batch_size=32
        height,weight=299,299
        input=tf.random_uniform( (batch_size,height,weight,3) )
        with slim.arg_scope(inception_v3_arg_scope()):
            logits,end_points=inception_v3(input,is_training=False)
 
        init=tf.global_variables_initializer()
        sess=tf.Session()
        # 将网络结构图写到文件中
        writer = tf.summary.FileWriter('logs/', sess.graph)
        sess.run(init)
        num_batches=100
        time_compute(sess,logits,'Forward')
 
 
 
if __name__=='__main__':
    main()